## Imports

In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mog_model import *
from utils.distributions import *
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice
from lattice.fields import SU2, SU3
from utils.dynamics import *
from utils.lattice_dynamics import *
from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, binarize, normal_kl, acl_spectrum, ESS
from utils.network import network, Linear, Sequential, Zip, Parallel, ScaleTanh
from utils.sampler import propose
from utils.notebook_utils import get_hmc_samples
from utils.data_utils import calc_avg_vals_errors, block_resampling, jackknife_err
from matplotlib import gridspec
from utils.plot_helper import errorbar_plot
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')

plt.rc('text', usetex=False)
plt.rcParams['errorbar.capsize'] = 0
%matplotlib notebook
%autoreload 2

In [2]:
sess = tf.InteractiveSession()

In [3]:
def restart_sess(sess):
    tf.reset_default_graph()
    sess.close()
    return tf.InteractiveSession()

## Create IsingLattice

In [34]:
sess = restart_sess(sess)

In [35]:
lattice = IsingLattice(3, 4)

In [36]:
samples = [lattice._randomize() for _ in range(10)]

In [37]:
batch_size = len(samples)
energy_fn = lattice.get_energy_function()
dynamics = Dynamics(x_dim=lattice.num_sites, energy_function=energy_fn, 
                    trajectory_length=2, eps=0.1, hmc=False, 
                    net_factory=network, eps_trainable=True,
                    use_temperature=True)

In [38]:
energies = lattice.calc_energy(samples, batch_size=len(samples))

In [39]:
energies

[24, 16, -16, 16, 16, 16, 24, 0, 72, 16]

In [40]:
x_shape = tuple([len(samples)] + list(lattice.sites.shape))
x = tf.placeholder(tf.float32, shape=x_shape, name='x')
z = tf.random_normal(tf.shape(x), name='z')

In [41]:
energy_fn = dynamics.energy(x, batch_size)

In [43]:
print(sess.run(energy_fn, feed_dict={x: samples, dynamics.temperature: 1.}))

[ 24.  16. -16.  16.  16.  16.  24.   0.  72.  16.]


In [48]:
kinetic = dynamics.kinetic(z)
_kinetic = sess.run(kinetic)

In [51]:
lattice.sites.shape

(4, 4, 4)

In [52]:
_kinetic.shape

(10, 4, 4)

In [53]:
z2 = sess.run(tf.square(z))

In [59]:
print(sess.run(tf.reduce_sum(tf.square(z), axis=z.shape)))

ValueError: Tried to convert 'reduction_indices' to a tensor and failed. Error: Argument must be a dense tensor: [TensorShape([Dimension(4), Dimension(4), Dimension(4)])] - got shape [1, 3], but wanted [1].

In [54]:
tf.reduce_sum(tf.square(z), axis=)

(10, 4, 4, 4)

In [ ]:
kinetic_ = tf.reduce_sum(tf.square(z), axis

In [44]:
H = dynamics.hamiltonian(x, z, batch_size)

ValueError: Dimensions must be equal, but are 10 and 4 for 'add_8960' (op: 'Add') with input shapes: [10], [10,4,4].

In [ ]:
print(sess.run(H, feed_dict={x: samples, dynamics.temperature: 1.}))

In [10]:
def eager_execution():
    dataset = tf.data.Dataset.from_tensor_slices(x)
    iterator = dataset.make_initializable_iterator()

    next_element = iterator.get_next()

    #dynamics_energy = dynamics.energy(, aux=None)
    sess.run(iterator.initializer, feed_dict={x: samples})
    for i in range(len(samples)):
        value = sess.run(next_element)
        energy = sess.run(dynamics.energy(value), 
                          feed_dict={dynamics.temperature: 1.})
        print(energy)
        #print(value.shape)
        #_energy = sess.run(dynamics.energy(, aux=None))
        #print(_energy)

    sess.run(iterator.initializer, feed_dict={x: samples})
    for i in range(len(samples)):
        next_elt = sess.run(next_element)
        _step = sess.run(dynamics._forward_step(next_elt, z, i, aux=None),
                         feed_dict={dynamics.temperature: 1.})
        print(_step)

In [95]:

hamiltonian = dynamics.hamiltonian(x, z, aux=None)

In [97]:
hamiltonian_out = sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature: 1.})

In [99]:
hamiltonian_out[0]

array([[24.517838 ,  4.7593355, 17.857069 , 17.039177 ],
       [28.00538  ,  1.9556428, 16.959648 , 18.82082  ],
       [25.566353 ,  1.1102458, 17.587902 , 17.814648 ],
       [28.372929 ,  0.5430869, 16.659918 , 17.761255 ]], dtype=float32)

In [63]:
step1 = dynamics._forward_step(x, z, 1, aux=None)

> /Users/saforem2/ANL/l2hmc/l2hmc/utils/network.py(125)__call__()
-> if x.dtype == tf.complex64:
(Pdb) x.shape
TensorShape([Dimension(None), Dimension(4), Dimension(4), Dimension(4)])
(Pdb) self.W.shape
TensorShape([Dimension(64), Dimension(100)])
(Pdb) W = tf.reshape(self.W, x.shape)
*** ValueError: Tried to convert 'shape' to a tensor and failed. Error: Cannot convert a partially known TensorShape to a Tensor: (?, 4, 4, 4)
(Pdb) ll
120  	    def __call__(self, x):
121  	        #  self.activations = tf.add(tf.matmul(x, self.W), self.b)
122  	        #self.W = tf.reshape(self.W, shape=tuple([x.shape] + [-1]))
123  	        import pdb
124  	        pdb.set_trace()
125  ->	        if x.dtype == tf.complex64:
126  	            self.activations = tf.add(complex_matmul(x, self.W),
127  	                                      tf.cast(self.b, tf.complex64))
128  	            #  self.activations = tf.add(complex_matmul(x, self.W),
129  	            #                            tf.cast(self.b, 

BdbQuit: 

In [6]:
Lx, _, px, output = propose(x, dynamics, do_mh_step=True)
Lz, _, pz, _ = propose(z, ldynamics, do_mh_step=True)

loss = tf.Variable(0., trainable=False, name='loss')
v1 = ((tf.reduce_sum(tf.square(x - Lx), axis=1) * px) + 1e-4)
v2 = ((tf.reduce_sum(tf.square(z - Lz), axis=1) * pz) + 1e-4)
scale = 0.1
loss = loss + scale * (tf.reduce_mean(1.0 / v1) + tf.reduce_mean(1.0 / v2))
loss = loss + ((-tf.reduce_mean(v1, name='v1')
                - tf.reduce_mean(v2, name='v2')) / scale)

ValueError: None values not supported.

In [7]:
%debug

> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py(424)make_tensor_proto()
    422   else:
    423     if values is None:
--> 424       raise ValueError("None values not supported.")
    425     # if dtype is provided, forces numpy array to be the type
    426     # provided if possible.

ipdb> ll
    350 @tf_export("make_tensor_proto")
    351 def make_tensor_proto(values, dtype=None, shape=None, verify_shape=False):
    352   """Create a TensorProto.
    353 
    354   Args:
    355     values:         Values to put in the TensorProto.
    356     dtype:          Optional tensor_pb2 DataType value.
    357     shape:          List of integers representing the dimensions of tensor.
    358     verify_shape:   Boolean that enables verification of a shape of values.
    359 
    360   Returns:
    361     A `TensorProto`. Depending on the type, it may contain data in the
    362     "tensor_content" attribute, which is

ipdb> up
> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/framework/constant_op.py(196)constant()
    194   tensor_value.tensor.CopyFrom(
    195       tensor_util.make_tensor_proto(
--> 196           value, dtype=dtype, shape=shape, verify_shape=verify_shape))
    197   dtype_value = attr_value_pb2.AttrValue(type=tensor_value.tensor.dtype)
    198   const_tensor = g.create_op(

ipdb> up
> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/framework/constant_op.py(217)_constant_tensor_conversion_function()
    215                                          as_ref=False):
    216   _ = as_ref
--> 217   return constant(v, dtype=dtype, name=name)
    218 
    219 

ipdb> up
> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/framework/ops.py(1094)internal_convert_to_tensor()
   1092 
   1093     if ret is None:
-> 1094       ret = conversion_func(value, dtype=dt

ipdb> args
self = <utils.dynamics.Dynamics object at 0xa40456198>
x = <tf.Tensor 'while/Identity:0' shape=(?, 64) dtype=float32>
v = <tf.Tensor 'while/Identity_1:0' shape=(?, 64) dtype=float32>
step = <tf.Tensor 'while/Identity_2:0' shape=() dtype=float32>
aux = None
ipdb> step


In [28]:
step1 = ldynamics._forward_step(x, z, 1, aux=None)

ValueError: Dimensions must be equal, but are 3 and 512 for 'mul_3840' (op: 'Mul') with input shapes: [?,2,4,4,4,4,3,3], [512,100].

In [ ]:
with tf.Session() as sess:
    print(sess.run(lattice.total_action(lattice.links)))

In [ ]:
energy = ldynamics.energy(x)
with tf.Session() as sess:
    print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature:1.}))

## LatticeDynamics testing

### Init tf.Session()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.close()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess.run(tf.global_variables_initializer())

### Create LatticeDynamics object

In [ ]:
_x_shape = tuple([None] + list(lattice.links.shape) + list(lattice.link_shape))
_x = tf.placeholder(dtype=tf.complex64, shape=_x_shape)

_links_samples = []
for _ in range(10):
    _links_sample = np.zeros(_x_shape[1:], dtype=np.complex64)
    for link in lattice.iter_links():
        _links_sample[link] = lattice.link_type.get_random_element()
    _links_samples.append(_links_sample)

In [ ]:
lattice.num_links

In [ ]:
_mask = sess.run(ldynamics.mask)

In [ ]:
_mask.reshape(tuple([-1] + list(lattice.links.shape)))

In [ ]:
m, _m = ldynamics._get_mask(0)

In [ ]:
np.array(links_samples).astype(np.complex64)

In [ ]:
links_samples = np.array(np.array(links_samples), dtype=np.complex64)

In [ ]:
kinetic = ldynamics.kinetic(z)
print(sess.run(kinetic), feed_dict={x: np.array(links_samples), dynamics.temperature: 1.})

In [ ]:
hamiltonian = ldynamics.hamiltonian(x, z)
sess.run(hamiltonian, feed_dict{x:})

In [ ]:
temp = np.array([1.0], dtype=np.float32)[0]
energy = ldynamics.energy(x)
print(sess.run(energy, feed_dict={x: links_samples, ldynamics.temperature: temp}))

In [ ]:
print(sess.run(energy, feed_dict={x: links_samples, ldynamics.temperature: temp}))

In [ ]:
print(sess.run(ldynamics.lattice.total_action()))
#ldynamics.lattice.total_action()

In [ ]:
forward_step = ldynamics._forward_step(x, z, 1, aux=None)

In [ ]:
%debug

In [ ]:
step1 = sess.run(forward_step, feed_dict={x: samples, ldynamics.temperature: 2.})

In [ ]:
x1, v1, log_jac_contrib1 = step1

In [ ]:
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: samples, dynamics.temperature:2.}))
print(sess.run(energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
print(sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(hamiltonian, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
grad_energy = dynamics.grad_energy(x)
print(sess.run(grad_energy, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(grad_energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
X, V, p_accept = dynamics.forward(x)

In [ ]:
_X = sess.run(X, feed_dict={x: samples, dynamics.temperature:2.})
print(_X.shape)

In [ ]:
_V = sess.run(V, feed_dict={x: samples, dynamics.temperature:2.})
print(_V.shape)

In [ ]:
_p_accept = sess.run(p_accept, feed_dict={x: samples, dynamics.temperature:2.})
print(_p_accept.shape)
print(_p_accept)